In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!7z x /content/drive/MyDrive/train.csv.7z
!7z x /content/drive/MyDrive/items.csv.7z
!7z x /content/drive/MyDrive/stores.csv.7z

In [ ]:
!pip install pyspark

**Using spark given dataset's vast size**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('sparkpy').getOrCreate()

In [ ]:
sales= spark.read.csv('/content/train.csv',header=True)
sales.show()

+---+----------+---------+--------+----------+
| id|      date|store_nbr|item_nbr|unit_sales|
+---+----------+---------+--------+----------+
|  0|2013-01-01|       25|  103665|       7.0|
|  1|2013-01-01|       25|  105574|       1.0|
|  2|2013-01-01|       25|  105575|       2.0|
|  3|2013-01-01|       25|  108079|       1.0|
|  4|2013-01-01|       25|  108701|       1.0|
|  5|2013-01-01|       25|  108786|       3.0|
|  6|2013-01-01|       25|  108797|       1.0|
|  7|2013-01-01|       25|  108952|       1.0|
|  8|2013-01-01|       25|  111397|      13.0|
|  9|2013-01-01|       25|  114790|       3.0|
| 10|2013-01-01|       25|  114800|       1.0|
| 11|2013-01-01|       25|  115267|       1.0|
| 12|2013-01-01|       25|  115611|       1.0|
| 13|2013-01-01|       25|  115693|       1.0|
| 14|2013-01-01|       25|  115720|       5.0|
| 15|2013-01-01|       25|  115850|       1.0|
| 16|2013-01-01|       25|  115891|       6.0|
| 17|2013-01-01|       25|  115892|      10.0|
| 18|2013-01-

In [ ]:
from pyspark.sql.types import DateType

sales = sales.withColumn('store_nbr', sales['store_nbr'].cast("integer"))
sales = sales.withColumn('date', sales['date'].cast(DateType()))
sales = sales.withColumn('item_id', sales['item_nbr'].cast("integer"))
sales = sales.withColumn('unit_sales', sales['unit_sales'].cast("float"))

sales.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- store_nbr: integer (nullable = true)
 |-- item_nbr: string (nullable = true)
 |-- unit_sales: float (nullable = true)
 |-- item_id: integer (nullable = true)



In [ ]:
sales.show(3)

+---+----------+---------+----------+-------+
| id|      date|store_nbr|unit_sales|item_id|
+---+----------+---------+----------+-------+
|  0|2013-01-01|       25|       7.0| 103665|
|  1|2013-01-01|       25|       1.0| 105574|
|  2|2013-01-01|       25|       2.0| 105575|
+---+----------+---------+----------+-------+
only showing top 3 rows



In [ ]:
items= spark.read.csv('/content/items.csv', header=True)

In [ ]:
items.show(3)

+--------+---------+-----+----------+
|item_nbr|   family|class|perishable|
+--------+---------+-----+----------+
|   96995|GROCERY I| 1093|         0|
|   99197|GROCERY I| 1067|         0|
|  103501| CLEANING| 3008|         0|
+--------+---------+-----+----------+
only showing top 3 rows



In [ ]:
items = items.withColumn('item_nbr', items['item_nbr'].cast("integer"))
items = items.withColumn('family', items['family'].cast("string"))
items = items.withColumn('class', items['class'].cast("integer"))
items = items.withColumn('perishable', items['perishable'].cast("integer"))

items.printSchema()

root
 |-- item_nbr: integer (nullable = true)
 |-- family: string (nullable = true)
 |-- class: integer (nullable = true)
 |-- perishable: integer (nullable = true)



In [ ]:
stores= spark.read.csv('/content/stores.csv', header=True)

In [ ]:
stores.show(3)

+---------+-----+---------+----+-------+
|store_nbr| city|    state|type|cluster|
+---------+-----+---------+----+-------+
|        1|Quito|Pichincha|   D|     13|
|        2|Quito|Pichincha|   D|     13|
|        3|Quito|Pichincha|   D|      8|
+---------+-----+---------+----+-------+
only showing top 3 rows



In [ ]:
stores = stores.withColumn('store_id', stores['store_nbr'].cast("integer"))
stores = stores.withColumn('city', stores['city'].cast("string"))
stores = stores.withColumn('state', stores['state'].cast("string"))
stores = stores.withColumn('type', stores['type'].cast("string"))
stores = stores.withColumn('cluster', stores['cluster'].cast("integer"))
stores.printSchema()

root
 |-- store_nbr: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- type: string (nullable = true)
 |-- cluster: integer (nullable = true)
 |-- store_id: integer (nullable = true)



In [ ]:
# Count of distinct cities present

stores.select('city').distinct().count()

22

In [ ]:
# Count of distinct stores present

sales.select('store_nbr').distinct().count()

54

In [1]:
# Count of distinct item types present

items.select('family').distinct().count()

33

In [ ]:
# Merging all 3 dataframes

sales_and_items = sales.join(items, sales.item_id==items.item_nbr, 'left')

In [ ]:
sales_and_items.show(3)

+---+----------+---------+----------+--------+------------+-----+----------+
| id|      date|store_nbr|unit_sales|item_nbr|      family|class|perishable|
+---+----------+---------+----------+--------+------------+-----+----------+
|  0|2013-01-01|       25|       7.0|  103665|BREAD/BAKERY| 2712|         1|
|  1|2013-01-01|       25|       1.0|  105574|   GROCERY I| 1045|         0|
|  2|2013-01-01|       25|       2.0|  105575|   GROCERY I| 1045|         0|
+---+----------+---------+----------+--------+------------+-----+----------+
only showing top 3 rows



In [ ]:
sales_items_stores = sales_and_items.join(stores, sales_and_items.store_nbr==stores.store_id, 'left')

In [ ]:
sales_items_stores = sales_items_stores.drop('store_id')

In [ ]:
sales_items_stores.show(truncate=True)

+---+----------+---------+----------+--------+-------------+-----+----------+-------+-----------+----+-------+
| id|      date|store_nbr|unit_sales|item_nbr|       family|class|perishable|   city|      state|type|cluster|
+---+----------+---------+----------+--------+-------------+-----+----------+-------+-----------+----+-------+
|  0|2013-01-01|       25|       7.0|  103665| BREAD/BAKERY| 2712|         1|Salinas|Santa Elena|   D|      1|
|  1|2013-01-01|       25|       1.0|  105574|    GROCERY I| 1045|         0|Salinas|Santa Elena|   D|      1|
|  2|2013-01-01|       25|       2.0|  105575|    GROCERY I| 1045|         0|Salinas|Santa Elena|   D|      1|
|  3|2013-01-01|       25|       1.0|  108079|    GROCERY I| 1030|         0|Salinas|Santa Elena|   D|      1|
|  4|2013-01-01|       25|       1.0|  108701|         DELI| 2644|         1|Salinas|Santa Elena|   D|      1|
|  5|2013-01-01|       25|       3.0|  108786|     CLEANING| 3044|         0|Salinas|Santa Elena|   D|      1|
|

In [ ]:
distinct_items_df = items.dropDuplicates(["family"])

In [ ]:
distinct_items_df.select('family', 'perishable').show()

+--------------------+----------+
|              family|perishable|
+--------------------+----------+
|      PREPARED FOODS|         1|
| HOME AND KITCHEN II|         0|
|          LADIESWEAR|         0|
|     LAWN AND GARDEN|         0|
|           GROCERY I|         0|
|           BABY CARE|         0|
|             PRODUCE|         1|
|          AUTOMOTIVE|         0|
|           BEVERAGES|         0|
|           HOME CARE|         0|
|        BREAD/BAKERY|         1|
|               BOOKS|         0|
|            LINGERIE|         0|
|         CELEBRATION|         0|
|          GROCERY II|         0|
|               DAIRY|         1|
|           MAGAZINES|         0|
|             SEAFOOD|         1|
|    LIQUOR,WINE,BEER|         0|
|  HOME AND KITCHEN I|         0|
|                DELI|         1|
|PLAYERS AND ELECT...|         0|
|             POULTRY|         1|
|        FROZEN FOODS|         0|
|SCHOOL AND OFFICE...|         0|
|       PERSONAL CARE|         0|
|            C

In [ ]:
distinct_states_df = stores.dropDuplicates(["state"])

In [ ]:
distinct_states_df.select('state').show()

+--------------------+
|               state|
+--------------------+
|              Manabi|
|            Cotopaxi|
|           Pichincha|
|          Chimborazo|
|              Guayas|
|                Loja|
|         Santa Elena|
|            Imbabura|
|              El Oro|
|               Azuay|
|             Bolivar|
|          Tungurahua|
|             Pastaza|
|Santo Domingo de ...|
|            Los Rios|
|          Esmeraldas|
+--------------------+



In [ ]:
distinct_city_df = stores.dropDuplicates(["city"])

In [ ]:
distinct_states_df.select('state').show()

+--------------------+
|               state|
+--------------------+
|              Manabi|
|            Cotopaxi|
|           Pichincha|
|          Chimborazo|
|              Guayas|
|                Loja|
|         Santa Elena|
|            Imbabura|
|              El Oro|
|               Azuay|
|             Bolivar|
|          Tungurahua|
|             Pastaza|
|Santo Domingo de ...|
|            Los Rios|
|          Esmeraldas|
+--------------------+



In [ ]:
from pyspark.sql.functions import col

In [ ]:
items = distinct_items_df.select('family').collect()

In [ ]:
items = [item[0] for item in items]

In [ ]:
items

['PREPARED FOODS',
 'HOME AND KITCHEN II',
 'LADIESWEAR',
 'LAWN AND GARDEN',
 'GROCERY I',
 'BABY CARE',
 'PRODUCE',
 'AUTOMOTIVE',
 'BEVERAGES',
 'HOME CARE',
 'BREAD/BAKERY',
 'BOOKS',
 'LINGERIE',
 'CELEBRATION',
 'GROCERY II',
 'DAIRY',
 'MAGAZINES',
 'SEAFOOD',
 'LIQUOR,WINE,BEER',
 'HOME AND KITCHEN I',
 'DELI',
 'PLAYERS AND ELECTRONICS',
 'POULTRY',
 'FROZEN FOODS',
 'SCHOOL AND OFFICE SUPPLIES',
 'PERSONAL CARE',
 'CLEANING',
 'HARDWARE',
 'PET SUPPLIES',
 'BEAUTY',
 'EGGS',
 'HOME APPLIANCES',
 'MEATS']

In [ ]:
cities = distinct_city_df.select('city').collect()

In [ ]:
cities = [city[0] for city in cities]

In [ ]:
cities

['Quevedo',
 'Cuenca',
 'Guaranda',
 'Santo Domingo',
 'Playas',
 'Puyo',
 'Quito',
 'Manta',
 'Latacunga',
 'Guayaquil',
 'Loja',
 'Ibarra',
 'El Carmen',
 'Ambato',
 'Machala',
 'Daule',
 'Cayambe',
 'Salinas',
 'Libertad',
 'Babahoyo',
 'Riobamba',
 'Esmeraldas']

In [ ]:
states = distinct_states_df.select('state').collect()

In [ ]:
states = [state[0] for state in states]

In [ ]:
states

['Manabi',
 'Cotopaxi',
 'Pichincha',
 'Chimborazo',
 'Guayas',
 'Loja',
 'Santa Elena',
 'Imbabura',
 'El Oro',
 'Azuay',
 'Bolivar',
 'Tungurahua',
 'Pastaza',
 'Santo Domingo de los Tsachilas',
 'Los Rios',
 'Esmeraldas']